# Python Codes - Part 2 - Hyperparameter Grid-Search 
(NN Model Outputs: "BOD", "COD", "NH3_N")

This Python code notebook is for the paper title "Quantitating Wastewater Characteristic Parameters Using Neural Network Regression Modeling on Spectral Reflectance" by Fortela, D.L.B. et al. (2023).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
 
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import  train_test_split
 
import tensorflow as tf
from tensorflow.keras import regularizers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression

In [ ]:
#Load data

data = pd.read_csv("peerj-07-8255-s001_xlsx_cut_NotInfluentWW_v2_CSV.csv")

X = data.values[1:,7:].astype('float32')
y = data[["BOD", "COD", "NH3_N"]].values[1:].astype("float32")

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=79)
 
scaler = MinMaxScaler()
scaler.fit(X_train)
Xs_train = scaler.transform(X_train)
Xs_test = scaler.transform(X_test)

In [ ]:
data.head()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adadelta

def make_regression_ann(activation1='relu', activation2='relu', activation3='linear', optimizer='Adam', learn_rate=0.0001, H1 = 64, H2 = 32):

    model = Sequential()
    model.add(Dense(units=H1, input_dim=1601, activation=activation1))
    model.add(Dense(units=H2, activation=activation2))
    model.add(Dense(3, activation=activation3))
    
    if optimizer=="Adam":
        opt = Adam(learning_rate=learn_rate)
        
    elif optimizer=='AMSgrad':
        opt = keras.optimizers.Adam(learning_rate=learn_rate, amsgrad=True)
        
    else :
        opt = keras.optimizers.SGD(learning_rate=learn_rate, momentum=0.2)
    
    model.compile(loss='mae', optimizer=opt)

    return model


## Define the grid-search parameters

In [ ]:
param_grid = {
    
    'activation1': ['relu', 'linear'],
    'activation2': ['relu', 'linear'],
    'activation3': ['linear'],
    'optimizer': ['Adam', 'Adadelta', 'SGD'],
    'learn_rate': [0.00001, 0.0001, 0.001],
    'H1' : [1600, 1000, 64],
    'H2' : [256, 64, 32],
}

## Run the grid-search

In [ ]:
grid_search = GridSearchCV(
    estimator=KerasRegressor(make_regression_ann, batch_size=10, verbose=1),
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=3,
)

In [ ]:
grid_search.fit(Xs_train, y_train, \
            epochs=5000, \
            validation_data=(Xs_test, y_test), \
            
            verbose=1
               )

## Print the best model based on MAE followed by all the runs implemented

In [ ]:
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Refitting the best model deteremined above:
Also, use  a learnig_rate scheduler to improve fitting

In [ ]:
model2 = tf.keras.Sequential([
tf.keras.layers.Dense(1600, activation='linear'),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dense(3, activation='linear')])

In [ ]:
# use learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=500,
    decay_rate=0.5)

# complie
model2.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule))

In [ ]:
refit_model2 = model2.fit(Xs_train, y_train, epochs=5000, \
            validation_data=(Xs_test, y_test), \
            verbose=1)

In [ ]:
with plt.style.context(('seaborn-paper')):
    fig, ax = plt.subplots(figsize=(3, 3))
 
    ax.plot(refit_model2.history['loss'], linewidth=2, label='Train loss', color='red')
    ax.plot(refit_model2.history['val_loss'], linewidth=2, label='Val. loss', linestyle='-.', color='blue')
 
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    ax.set_xlabel('Epoch',fontsize=12)
    ax.set_ylabel('MAE',fontsize=12)
    plt.legend(fontsize=12)
 
    plt.tight_layout()
plt.savefig('Training_MAE_loss.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()

In [ ]:

predictions = model2.predict(Xs_test)#.flatten()
rmse, mae, score = np.sqrt(mean_squared_error(y_test, predictions)), \
                   mean_absolute_error(y_test, predictions), r2_score(y_test, predictions)
print("R2: %5.3f, RMSE: %5.3f, MAE: %5.3f" %(score, rmse, mae))

In [ ]:
# see values of predictions on test dataset
predictions

In [ ]:
# see values of actual data on test dataset
y_test

In [ ]:
# Plot BOD test dataset

# subset the data
predicted_value = predictions[:,0]
true_value = y_test[:,0]

x_axis = true_value
y_axis = predicted_value

# create plot
plt.figure(figsize=(3,3))
p1 = max(max(true_value), max(true_value))
p2 = min(min(true_value), min(true_value))
plt.plot([0, p1], [0, p1], 'r-', label="Y = T")

plt.xticks(np.arange(0, max(x_axis), 2))
plt.yticks(np.arange(0, max(y_axis), 2))

plt.axis('equal')
plt.scatter(true_value, predicted_value, c='aqua', alpha=1, edgecolors='black', label="Y vs T")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# add reference line
#plt.axvline(x=25, color='black', ls='--', lw=1)

# add titles
plt.xlabel('T = True Values', fontsize=11)
plt.ylabel('Y = Predictions', fontsize=11)
plt.title("Test: BOD (mg/L)")


# calc r-squared
z = np.polyfit(x_axis, y_axis, 1)  
p = np.poly1d(z)  

# plot line fit
plt.plot(x_axis,p(x_axis),'b-.', lw=1, label="Y = f(T)") 
#plt.plot(x_axis, y_axis, 'og-', label=("y=%.6fx+(%.6f)"%(z[0],z[1]))) 

r_squared = r2_score(x_axis, y_axis)
plt.annotate("R^2 = {:.3f}".format(r_squared), (0, 8))


# annonate MAE
# import the module
from sklearn.metrics import mean_absolute_error as mae
# calculate MAE
error = mae(x_axis, y_axis)
plt.annotate("MAE = {:.3f}".format(error), (0, 7.0))


# show legend
plt.legend(loc="lower right")

plt.savefig('Test_BOD.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()

In [ ]:
# Plot COD test dataset

# subset the data
predicted_value = predictions[:,1]
true_value = y_test[:,1]

x_axis = true_value
y_axis = predicted_value

# create plot
plt.figure(figsize=(3,3))
p1 = max(max(true_value), max(true_value))
p2 = min(min(true_value), min(true_value))
plt.plot([0, p1], [0, p1], 'r-', label="Y = T")

plt.xticks(np.arange(0, max(x_axis), 20))
plt.yticks(np.arange(0, max(y_axis), 20))

plt.axis('equal')
plt.scatter(true_value, predicted_value, c='orange', alpha=1, edgecolors='black', label="Y vs T")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# add reference line
#plt.axvline(x=100, color='black', ls='--', lw=1)

# add titles
plt.xlabel('T = True Values', fontsize=11)
plt.ylabel('Y = Predictions', fontsize=11)
plt.title("Test: COD (mg/L)")


# calc r-squared
z = np.polyfit(x_axis, y_axis, 1)  
p = np.poly1d(z)  

# plot line fit
plt.plot(x_axis,p(x_axis),'b-.', lw=1, label="Y = f(T)") 
#plt.plot(x_axis, y_axis, 'og-', label=("y=%.6fx+(%.6f)"%(z[0],z[1]))) 

r_squared = r2_score(x_axis, y_axis)
plt.annotate("R^2 = {:.3f}".format(r_squared), (0, 120))


# annonate MAE
# import the module
from sklearn.metrics import mean_absolute_error as mae
# calculate MAE
error = mae(x_axis, y_axis)
plt.annotate("MAE = {:.3f}".format(error), (0, 110))


# show legend
plt.legend(loc="lower right")

plt.savefig('Test_COD.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()

In [ ]:
# Plot NH3NN test dataset

# subset the data
predicted_value = predictions[:,2]
true_value = y_test[:,2]

x_axis = true_value
y_axis = predicted_value

# create plot
plt.figure(figsize=(3,3))
p1 = max(max(true_value), max(true_value))
p2 = min(min(true_value), min(true_value))
plt.plot([0, p1], [0, p1], 'r-', label="Y = T")

plt.xticks(np.arange(0, max(x_axis), 0.5))
plt.yticks(np.arange(0, max(y_axis), 0.5))

plt.axis('equal')
plt.scatter(true_value, predicted_value, c='lime', alpha=1, edgecolors='black', label="Y vs T")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# add reference line
#plt.axvline(x=10, color='black', ls='--', lw=1)

# add titles
plt.xlabel('T = True Values', fontsize=11)
plt.ylabel('Y = Predictions', fontsize=11)
plt.title("Test: NH3-N (mg/L)")

# calc r-squared
z = np.polyfit(x_axis, y_axis, 1)  
p = np.poly1d(z)  

# plot line fit
plt.plot(x_axis,p(x_axis),'b-.', lw=1, label="Y = f(T)") 
#plt.plot(x_axis, y_axis, 'og-', label=("y=%.6fx+(%.6f)"%(z[0],z[1]))) 

r_squared = r2_score(x_axis, y_axis)
plt.annotate("R^2 = {:.3f}".format(r_squared), (-0.15, 1.1))


# annonate MAE
# import the module
from sklearn.metrics import mean_absolute_error as mae
# calculate MAE
error = mae(x_axis, y_axis)
plt.annotate("MAE = {:.3f}".format(error), (-0.15, 0.9))

# show legend
plt.legend(loc="upper left")

plt.savefig('Test_NH3-N.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()

In [ ]:
train_predictions = model2.predict(Xs_train)#.flatten()
rmse, mae, score = np.sqrt(mean_squared_error(y_train, train_predictions)), \
                   mean_absolute_error(y_train, train_predictions), r2_score(y_train, train_predictions)
print("R2: %5.3f, RMSE: %5.3f, MAE: %5.3f" %(score, rmse, mae))

In [ ]:
# Plot BOD training dataset

# subset the data
predicted_value = train_predictions[:,0]
true_value = y_train[:,0]

x_axis = true_value
y_axis = predicted_value

# create plot
plt.figure(figsize=(3,3))
p1 = max(max(true_value), max(true_value))
p2 = min(min(true_value), min(true_value))
plt.plot([0, p1], [0, p1], 'r-', label="Y = T")

plt.xticks(np.arange(0, max(x_axis), 2))
plt.yticks(np.arange(0, max(y_axis), 2))

plt.axis('equal')
plt.scatter(true_value, predicted_value, c='aqua', alpha=1, edgecolors='black', label="Y vs T")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


# add reference line
#plt.axvline(x=1, color='black', ls='--', lw=1)

# add titles
plt.xlabel('T = True Values', fontsize=11)
plt.ylabel('Y = Predictions', fontsize=11)
plt.title("Train: BOD (mg/L)")

# calc r-squared
z = np.polyfit(x_axis, y_axis, 1)  
p = np.poly1d(z)  

# plot line fit
plt.plot(x_axis,p(x_axis),'b-.', lw=1, label="Y = f(T)") 
#plt.plot(x_axis, y_axis, 'og-', label=("y=%.6fx+(%.6f)"%(z[0],z[1]))) 

r_squared = r2_score(x_axis, y_axis)
plt.annotate("R^2 = {:.3f}".format(r_squared), (4.5, 1.0))


# annonate MAE
# import the module
from sklearn.metrics import mean_absolute_error as mae
# calculate MAE
error = mae(x_axis, y_axis)
plt.annotate("MAE = {:.3f}".format(error), (4.5, 0.25))


# show legend
plt.legend(loc="upper left")

plt.savefig('Train_BOD.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()

In [ ]:
# Plot COD trainig dataset

# subset the data
predicted_value = train_predictions[:,1]
true_value = y_train[:,1]

x_axis = true_value
y_axis = predicted_value

# create plot
plt.figure(figsize=(3,3))
p1 = max(max(true_value), max(true_value))
p2 = min(min(true_value), min(true_value))
plt.plot([0, p1], [0, p1], 'r-', label="Y = T")

plt.xticks(np.arange(0, max(x_axis), 20))
plt.yticks(np.arange(0, max(y_axis), 20))

plt.axis('equal')
#plt.scatter(true_value, predicted_value, c='crimson', alpha=1, edgecolors='black')
plt.scatter(true_value, predicted_value, c='orange', alpha=1, edgecolors='black', label="Y vs T")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


# add reference line
#plt.axvline(x=150, color='black', ls='--', lw=1)


# add titles
plt.xlabel('T = True Values', fontsize=11)
plt.ylabel('Y = Predictions', fontsize=11)
plt.title("Train: COD (mg/L)")

# calc r-squared
z = np.polyfit(x_axis, y_axis, 1)  
p = np.poly1d(z)  

# plot line fit
plt.plot(x_axis,p(x_axis),'b-.', lw=1, label="Y = f(T)") 
#plt.plot(x_axis, y_axis, 'og-', label=("y=%.6fx+(%.6f)"%(z[0],z[1]))) 

r_squared = r2_score(x_axis, y_axis)
plt.annotate("R^2 = {:.3f}".format(r_squared), (10, 120))

# annonate MAE
# import the module
from sklearn.metrics import mean_absolute_error as mae
# calculate MAE
error = mae(x_axis, y_axis)
plt.annotate("MAE = {:.3f}".format(error), (10, 110))


# show legend
plt.legend(loc="lower right")


plt.savefig('Train_COD.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()

In [ ]:
# Plot NH3-N training dataset

# subset the data
predicted_value = train_predictions[:,2]
true_value = y_train[:,2]

x_axis = true_value
y_axis = predicted_value

# create plot
plt.figure(figsize=(3,3))
p1 = max(max(true_value), max(true_value))
p2 = min(min(true_value), min(true_value))
plt.plot([0, p1], [0, p1], 'r-', label="Y = T")

plt.xticks(np.arange(0, max(x_axis), 0.5))
plt.yticks(np.arange(0, max(y_axis), 0.5))

plt.axis('equal')
plt.scatter(true_value, predicted_value, c='lime', alpha=1, edgecolors='black', label="Y vs T")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# add reference line
#plt.axvline(x=10, color='black', ls='--', lw=1)

# add titles
plt.xlabel('T = True Values', fontsize=11)
plt.ylabel('Y = Predictions', fontsize=11)
plt.title("Train: NH3-N (mg/L)")


# calc r-squared
z = np.polyfit(x_axis, y_axis, 1)  
p = np.poly1d(z)  

# plot line fit
plt.plot(x_axis,p(x_axis),'b-.', lw=1, label="Y = f(T)") 
#plt.plot(x_axis, y_axis, 'og-', label=("y=%.6fx+(%.6f)"%(z[0],z[1]))) 

r_squared = r2_score(x_axis, y_axis)
plt.annotate("R^2 = {:.3f}".format(r_squared), (0.0, 1.45))

# annonate MAE
# import the module
from sklearn.metrics import mean_absolute_error as mae
# calculate MAE
error = mae(x_axis, y_axis)
plt.annotate("MAE = {:.3f}".format(error), (0.0, 1.35))


# show legend
plt.legend(loc="lower right")

plt.savefig('Train_NH3-N.svg',dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.show()
